In [1]:
#to use the language model, make sure you've unzipped the languageModel.tar.gz file
#and have compiled the code in the LanguageModelDecoder folder
baseDir = '/home/s2/nlp002/pj_data'

In [ ]:

import os
from glob import glob
from pathlib import Path
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
from omegaconf import OmegaConf
import tensorflow as tf
from neuralDecoder.neuralSequenceDecoder import NeuralSequenceDecoder

In [17]:
#evaluate the RNN on the test partition and competitionHoldOut partition
testDirs = ['test','competitionHoldOut']
trueTranscriptions = [[],[]]
decodedTranscriptions = [[],[]]
for dirIdx in range(2):
    ckptDir = baseDir + '/derived/real_early_stop/baselineRelease'

    args = OmegaConf.load(os.path.join(ckptDir, 'args.yaml'))
    args['loadDir'] = ckptDir
    args['mode'] = 'infer'
    args['loadCheckpointIdx'] = None

    for x in range(len(args['dataset']['datasetProbabilityVal'])):
        args['dataset']['datasetProbabilityVal'][x] = 0.0

    for sessIdx in range(4,19):
        args['dataset']['datasetProbabilityVal'][sessIdx] = 1.0
        args['dataset']['dataDir'][sessIdx] = baseDir+'/derived/tfRecords'
    args['testDir'] = testDirs[dirIdx]

    # Initialize model
    tf.compat.v1.reset_default_graph()
    nsd = NeuralSequenceDecoder(args)
    nsd._load_model(ckptDir + "/6200_model_weights.h5")
    # Inference
    out = nsd.inference()
    
    # print("[[[[[[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]]]]]]")
    # print(out["decodedSentences"])
    # print("[[[[[[[[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]]]]]]]]")
    # print(out["targetLength"])

    trueTranscriptions[dirIdx] = out['targetSentences']
    decodedTranscriptions[dirIdx] = out["decodedSentences"]


Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.04.28/train
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.04.28/test
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.05/train
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.05/test
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.17/train
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.17/test
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.19/train
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.19/test
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.24/train
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.24/test
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.26/train
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.05.26/test
Load data from /home/s2/nlp002/pj_data/derived/tfRecords/t12.2022.06.0

ValueError: Weight count mismatch for layer #0 (named encoder_1 in the current model, encoder in the save file). Layer expects 97 weight(s). Received 129 saved weight(s)

In [37]:
print(trueTranscriptions)
print(decodedTranscriptions)

[[["it's really hard to find something that works", 'and i loved ths shining', 'i do have a friend that ran', 'just way in the back', 'actually it makes sense to a certain extent', 'assure the government that he was employed', "they feel good about what they're doing", 'besides its an invasion of personal privacy', 'so you travel a lot', "so we're going to wait", 'any kind of a good environment to live in', 'the best of android', "i don't know what you think about the point", 'i believe it costs about ten dollars', 'what in the world drugs are you on to', 'android tv box', 'a couple of times', 'i had the brakes done on it', 'which i use as a reference and has been real handy', 'i know when i was working', "it's a package deal", 'i have seen it', 'the measures already passed a senate committee', 'they told me that this was a topic', 'in the red square or wherever said', 'they did a lot of play work and stuff', 'you can gain too much weight', 'recent legislation', "i'm a car buff too", '

In [43]:
print(trueTranscriptions[0][0])
print(decodedTranscriptions[0][0])

["it's really hard to find something that works", 'and i loved ths shining', 'i do have a friend that ran', 'just way in the back', 'actually it makes sense to a certain extent', 'assure the government that he was employed', "they feel good about what they're doing", 'besides its an invasion of personal privacy', 'so you travel a lot', "so we're going to wait", 'any kind of a good environment to live in', 'the best of android', "i don't know what you think about the point", 'i believe it costs about ten dollars', 'what in the world drugs are you on to', 'android tv box', 'a couple of times', 'i had the brakes done on it', 'which i use as a reference and has been real handy', 'i know when i was working', "it's a package deal", 'i have seen it', 'the measures already passed a senate committee', 'they told me that this was a topic', 'in the red square or wherever said', 'they did a lot of play work and stuff', 'you can gain too much weight', 'recent legislation', "i'm a car buff too", 'jo

In [47]:
from jiwer import wer
from jiwer import cer
# print(len(decodedTranscriptions))
# print(len(decodedTranscriptions[0]))
# print(len(decodedTranscriptions[0][0]))
#get word error rate and phoneme error rate for the test set (cer is actually phoneme error rate here)
decoded_cer = 0
cnt = 0
for idx in range(len(decodedTranscriptions[0])):
    for ix in range(len(decodedTranscriptions[0][idx])):
        decoded_cer += cer(decodedTranscriptions[0][idx][ix], trueTranscriptions[0][idx][ix])
        cnt += 1
decoded_cer /= cnt

decoded_wer = 0
cnt = 0
for idx in range(len(decodedTranscriptions[0])):
    for ix in range(len(decodedTranscriptions[0][idx])):
        decoded_wer += wer(decodedTranscriptions[0][idx][ix],trueTranscriptions[0][idx][ix])
        cnt += 1
decoded_wer /= cnt

print(decoded_cer)
print(decoded_wer)

4.166666666666667e-05
0.00018518518518518518


In [48]:
#print the sentence predictions for the test set
print(decodedTranscriptions[0])

[["it's really hard to find something that works", 'and i loved ths shining', 'i do have a friend that ran', 'just way in the back', 'actually it makes sense to a certain extent', 'assure the government that he was employed', "they feel good about what they're doing", 'besides its an invasion of personal privacy', 'so you travel a lot', "so we're going to wait", 'any kind of a good environment to live in', 'the best of android', "i don't know what you think about the point", 'i believe it costs about ten dollars', 'what in the world drugs are you on to', 'android tv box', 'a couple of times', 'i had the brakes done on it', 'which i use as a reference and has been real handy', 'i know when i was working', "it's a package deal", 'i have seen it', 'the measures already passed a senate committee', 'they told me that this was a topic', 'in the red square or wherever said', 'they did a lot of play work and stuff', 'you can gain too much weight', 'recent legislation', "i'm a car buff too", 'j

In [7]:
#print the predictions for the competition hold-out set (labels are unreleased)
print(decodedTranscriptions[1])

["i'm initially from colorado", "if i had money i'd be safe to buy a known", 'i raced in york', 'i thought the topic was pouring', 'what every drive up there', "you don't get evolve", 'i hope you enjoy', "we don't do a", 'a voice from the crowd', 'i hope you enjoy my blog', 'i never have it', 'maybe even too far', "it's had that russians were quite some time", 'quick animus or suit or dress', 'manufactured by nineteen seventy five', 'the water takes the recession', "because that's all she talked about", 'did you hear an piece about that', 'i play in several company sources', "i've heard on the news", 'maybe but in a little bit fit', 'and i guess it was late me', 'in just cause', 'the word of the confusion', 'and it evoked an emotional response', 'you want to buy something', "that's basically what we do", 'i guess a lot of people do', 'a lot of people have real water', 'it seems like he would develop pride', "but you're not", 'they make a collection on monday for yard waste', 'several o

In [49]:
#format the predictions for competition submission. This generates a .txt file that can be submitted.
with open('baselineCompetitionSubmission.txt', 'w') as f:
    for x in range(len(decodedTranscriptions[1])):
        for idx in range(len(decodedTranscriptions[1][x])):
            f.write(decodedTranscriptions[1][x][idx]+'\n')